# Import

In [1]:
import torch

# IoU

In [2]:
from yolo.src.metrics import iou

In [3]:
pred = torch.tensor([
    [3, 3, 7, 7],
    [1, -1, 3, 2],
    [1, 1, 2, 2]
])

true = torch.tensor([
    [4, 4, 8, 8],
    [2, -1, 3, 2],
    [3, 3, 4, 4]
])

In [4]:
iou(pred, true, format='pascal_voc')

tensor([0.3913, 0.5000, 0.0000])

# Non max suppresion

In [11]:
from collections import defaultdict

In [5]:
# pred = [
#     [class, prob, x1, y1, x2, y2]
# ]

In [41]:
predictions = [
    [1, 0.6, 4, 4, 7, 7],
    [2, 0.7, 3, 3, 5, 5],
    [2, 0.8, 2, 2, 3, 3],
    [2, 0.6, 3, 3, 4.9, 4.9]
]
iou_threshold = 0.5
prob_threshold = 0.3
box_format = 'pascal_voc'